In [9]:
from decimal import Decimal, getcontext

from vector import Vector

getcontext().prec = 30


class Plane(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 3

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Plane.first_nonzero_index(n)
            initial_coefficient = n[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Plane.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Plane.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output


    def is_parallel(self, p):
        return self.normal_vector.isParallel(p.normal_vector)
    
    def is_same(self, p):
        if self.normal_vector.is_zero():
            if not p.normal_vector.is_zero():
                return False
            else:
                diff = self.constant_term - p.constant_term
                return MyDecimal(diff).is_near_zero()
        elif p.normal_vector.is_zero():
            return False
            
        if not self.is_parallel(p):
            return False
        v1 = self.basepoint
        v2 = p.basepoint
        v = v1 - v2
        return (v.isOrthogonal(self.normal_vector) and v.isOrthogonal(p.normal_vector))

    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Plane.NO_NONZERO_ELTS_FOUND_MSG)


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps


In [5]:
p1 = Plane(Vector([1,2,3]), 10)
print(p1)

x_1 + 2x_2 + 3x_3 = 10


In [30]:
p1 = Plane(Vector([-0.412, 3.806, 0.728]), -3.46)
p2 = Plane(Vector([1.03, -9.515, -1.82]), 8.65)
print("Parallel: %s" % p1.is_parallel(p2))
print("Same: %s" % p1.is_same(p2))

print('')
p1 = Plane(Vector([2.611, 5.528, 0.283]), 4.6)
p2 = Plane(Vector([7.715, 8.306, 5.342]), 3.76)
print("Parallel: %s" % p1.is_parallel(p2))
print("Same: %s" % p1.is_same(p2))

print('')
p1 = Plane(Vector([-7.926, 8.625, -7.212]),-7.952)
p2 = Plane(Vector([-2.642, 2.875, -2.404]), -2.443)
print("Parallel: %s" % p1.is_parallel(p2))
print("Same: %s" % p1.is_same(p2))


Parallel: True
Same: True

Parallel: False
Same: False

Parallel: True
Same: False


In [25]:
print(-7.926/-2.642)
print(8.625/2.875)
print(-7.217/-2.404)
print(-7.952/-2.443)
print(2.404*3)
print(1.03/-0.412)
print(-9.515/3.806)
print(-1.82/0.728)
print(8.65/-3.46)



3.0
3.0
3.00207986689
3.25501432665
7.212
-2.5
-2.5
-2.5
-2.5
